# Informal tests of sqlalchemy wrapper

In [1]:
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_headless

pg = pglite_headless()
pg.ready()

postgresWidget(headless=True, response={'status': 'initialising'})

In [2]:
%%pglite_magic -w pg -r
CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY ,
            name TEXT NOT NULL
)


{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 0},
 'response_type': 'single'}

In [3]:
%%pglite_magic -w pg -r
        CREATE TABLE IF NOT EXISTS orders (
            id SERIAL PRIMARY KEY,
            user_id INTEGER NOT NULL,
            product TEXT NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
        )

{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 0},
 'response_type': 'single'}

In [4]:
%%pglite_magic -w pg -r
INSERT INTO users (id,name) VALUES (2, 'Alice');

{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 1},
 'response_type': 'single'}

In [5]:
from jupyter_anywidget_pglite.sqlalchemy_api import create_engine
import pandas as pd

conn2 = create_engine(pg)
pd.read_sql("SELECT * FROM users;", conn2)

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'SELECT * FROM users;' AND table_schema = 'public';
        
Executing query: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'SELECT * FROM users;' AND table_schema = 'public';
        
Executing statement of type: <class 'str'>
Statement is already a string: SELECT * FROM users;
Executing query: SELECT * FROM users;


,id,name
0,2,Alice


In [6]:
from sqlalchemy import inspect

inspector = inspect(conn2)

In [7]:
inspector.get_table_names()

Executing statement of type: <class 'str'>
Statement is already a string: SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'public';
Executing query: SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'public';


['users', 'orders']

In [8]:
inspector.has_table("users"), inspector.has_table("users2"),

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'users' AND table_schema = 'public';
        
Executing query: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'users' AND table_schema = 'public';
        


Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'users2' AND table_schema = 'public';
        
Executing query: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'users2' AND table_schema = 'public';
        


(True, False)

In [9]:
inspector.get_columns("users")

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT column_name, data_type, is_nullable, column_default 
        FROM information_schema.columns 
        WHERE table_name = 'users'
        AND table_schema = 'public';
        
Executing query: 
        SELECT column_name, data_type, is_nullable, column_default 
        FROM information_schema.columns 
        WHERE table_name = 'users'
        AND table_schema = 'public';
        


[{'name': 'id',
  'type': Integer(),
  'nullable': False,
  'default': "nextval('users_id_seq'::regclass)"},
 {'name': 'name', 'type': String(), 'nullable': False, 'default': None}]

In [10]:
inspector.get_pk_constraint("users")

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT kcu.column_name
        FROM information_schema.table_constraints AS tc
        JOIN information_schema.key_column_usage AS kcu 
            ON tc.constraint_name = kcu.constraint_name
            AND tc.table_schema = kcu.table_schema
        WHERE tc.constraint_type = 'PRIMARY KEY' 
            AND kcu.table_name = 'users'
            AND kcu.table_schema = 'public';
        
Executing query: 
        SELECT kcu.column_name
        FROM information_schema.table_constraints AS tc
        JOIN information_schema.key_column_usage AS kcu 
            ON tc.constraint_name = kcu.constraint_name
            AND tc.table_schema = kcu.table_schema
        WHERE tc.constraint_type = 'PRIMARY KEY' 
            AND kcu.table_name = 'users'
            AND kcu.table_schema = 'public';
        


{'constrained_columns': ['id'], 'name': 'pk_users'}

In [11]:
inspector.get_foreign_keys("orders")

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT 
            kcu.column_name, 
            ccu.table_schema, 
            ccu.table_name, 
            ccu.column_name,
            tc.constraint_name
        FROM information_schema.table_constraints AS tc
        JOIN information_schema.key_column_usage AS kcu 
            ON tc.constraint_name = kcu.constraint_name
            AND tc.table_schema = kcu.table_schema
        JOIN information_schema.constraint_column_usage AS ccu 
            ON ccu.constraint_name = tc.constraint_name
            AND ccu.table_schema = tc.table_schema
        WHERE tc.constraint_type = 'FOREIGN KEY' 
            AND kcu.table_name = 'orders'
            AND kcu.table_schema = 'public';
        
Executing query: 
        SELECT 
            kcu.column_name, 
            ccu.table_schema, 
            ccu.table_name, 
            ccu.column_name,
            tc.constraint_name
        FROM information_schema.table_

[{'name': 'fk_orders_id',
  'constrained_columns': ['id'],
  'referred_schema': 'public',
  'referred_table': 'users',
  'referred_columns': ['orders_user_id_fkey']}]

In [12]:
inspector.get_indexes("users")

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT indexname, indexdef
        FROM pg_indexes
        WHERE tablename = 'users' AND schemaname = 'public';
        
Executing query: 
        SELECT indexname, indexdef
        FROM pg_indexes
        WHERE tablename = 'users' AND schemaname = 'public';
        


[{'name': 'users_pkey', 'column_names': ['id'], 'unique': True}]

In [13]:
inspector.get_unique_constraints("orders")

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT tc.constraint_name, kcu.column_name
        FROM information_schema.table_constraints AS tc
        JOIN information_schema.key_column_usage AS kcu
            ON tc.constraint_name = kcu.constraint_name
        WHERE tc.constraint_type = 'UNIQUE'
            AND tc.table_name = 'orders'
            AND tc.table_schema = 'public'
        ORDER BY tc.constraint_name, kcu.ordinal_position;
        
Executing query: 
        SELECT tc.constraint_name, kcu.column_name
        FROM information_schema.table_constraints AS tc
        JOIN information_schema.key_column_usage AS kcu
            ON tc.constraint_name = kcu.constraint_name
        WHERE tc.constraint_type = 'UNIQUE'
            AND tc.table_name = 'orders'
            AND tc.table_schema = 'public'
        ORDER BY tc.constraint_name, kcu.ordinal_position;
        


[]

In [14]:
inspector.get_check_constraints("orders")

Executing statement of type: <class 'str'>
Statement is already a string: 
        SELECT conname, pg_get_expr(conbin, conrelid) AS consrc
        FROM pg_constraint
        WHERE conrelid = (
            SELECT oid
            FROM pg_class
            WHERE relname = 'orders'
            AND relnamespace = (
                SELECT oid
                FROM pg_namespace
                WHERE nspname = 'public'
            )
        )
        AND contype = 'c';
        
Executing query: 
        SELECT conname, pg_get_expr(conbin, conrelid) AS consrc
        FROM pg_constraint
        WHERE conrelid = (
            SELECT oid
            FROM pg_class
            WHERE relname = 'orders'
            AND relnamespace = (
                SELECT oid
                FROM pg_namespace
                WHERE nspname = 'public'
            )
        )
        AND contype = 'c';
        


[]